In [1]:
import networkx as nx
import node2vec
from node2vec import Node2Vec

from utils import *

%load_ext autoreload
%autoreload 2

In [2]:
G = nx.read_gpickle('data/network.gpickle')
G.number_of_nodes(), G.number_of_edges()

(60789, 75647)

To search for node embeddings I use node2vec approach. I tried to use network SVD from network-sklearn, but it didn't work well (I got very low scaled vectors with values about 1e-20). node2vec gives more pleasant embeddings in terms of using dot product or cosine similarity. By the way it's not so fast (model trains for 3,5 minutes for graph with 60k vertices and 75k edges with some hyperparameters tuning)

In [7]:
def generate_node_embeddings(graph: nx.Graph):
    model = Node2Vec(graph, dimensions=128, num_walks=30, walk_length=30, temp_folder='temp/', workers=2)
    result = model.fit(window=5, min_count=1, batch_words=5)
    return result

In [8]:
%%time
embeddings = generate_node_embeddings(G)

Computing transition probabilities:   0%|          | 0/60789 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 15/15 [03:20<00:00, 13.39s/it]


CPU times: user 46min 51s, sys: 2min 21s, total: 49min 13s
Wall time: 24min 59s


In [9]:
for idx, e in enumerate(G.edges()):
    if idx == 5:
        break
    print(e)

(0, 1)
(0, 24)
(0, 350192)
(1, 2)
(2, 3)


In [10]:
embeddings.wv.most_similar('0')

[('1', 0.9895632266998291),
 ('350192', 0.9811784029006958),
 ('24', 0.9749113321304321),
 ('2', 0.9552937746047974),
 ('350191', 0.9526708722114563),
 ('200921', 0.9499428272247314),
 ('200923', 0.9229863882064819),
 ('332388', 0.9122338891029358),
 ('3', 0.9114905595779419),
 ('350190', 0.9056087732315063)]

In [11]:
embeddings.save('model_data/embeddings.data')

In [12]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(int(node), x=embeddings.wv[str(node)].copy())

In [13]:
nx.readwrite.write_gpickle(G, 'data/network.gpickle')